## Customer churn prediction

#### Dataset description
- This dataset is randomly collected from an Iranian telecom company database over a period of 12 months. 
- A total of 3150 rows of data, each representing a customer, bear information for 13 columns.


#### Columns description
- **Call Failures**: number of call failures
- **Complains**: binary (0: No complaint, 1: complaint)
- **Subscription**: Length: total months of subscription
- **Charge Amount**: Ordinal attribute (0: lowest amount, 9: highest amount)
- **Seconds of Use**: total seconds of calls
- **Frequency of use**: total number of calls
- **Frequency of SMS**: total number of text messages
- **Distinct Called Numbers**: total number of distinct phone calls 
- **Age Group**: ordinal attribute (1: younger age, 5: older age)
- **Tariff Plan**: binary (1: Pay as you go, 2: contractual)
- **Status**: binary (1: active, 2: non-active)
- **Churn**: binary (1: churn, 0: non-churn)
- **Customer Value**: The calculated value of customer

#### Imports

In [ ]:
import math
import warnings
import pandas as pd
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,  GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE

#### Declaring constants / Project config

In [ ]:
INPUT_PATH = './input/'
OUTPUT_PATH = './output/'

INPUT_FILENAME = 'raw.csv'

LABELS_DICT = {
    'call_failure': 'Call Failure',
    'complains': 'Complains',
    'subscription_length': 'Subscription Length',
    'charge_amount': 'Charge Amount',
    'seconds_of_use': 'Seconds of Use',
    'frequency_of_use': 'Frequency of Use',
    'frequency_of_sms': 'Frequency of SMS',
    'distinct_called_numbers': 'Distinct Called Numbers',
    'age_group': 'Age Group',
    'tariff_plan': 'Tariff Plan',
    'status': 'Status',
    'age': 'Age',
    'customer_value': 'Customer Value',
    'churn': 'Churn'
}

target_col = 'churn'

px.defaults.template = 'plotly_dark'

# pio.renderers.default = "notebook" 

# za graph preview na Github-u
png_renderer = pio.renderers["png"]
png_renderer.width = 1400
png_renderer.height = 500
pio.renderers.default = "png" 

warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

#### Reading input data

In [ ]:
df = pd.read_csv(INPUT_PATH + INPUT_FILENAME)

#### Previewing data attributes

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

#### Profiling data (ydata_profiler)

In [ ]:
from ydata_profiling import ProfileReport

# profile = ProfileReport(df, title="Profiling Report")

# profile.to_file(f'{OUTPUT_PATH}profiling_report.html')

#### Renaming columns to snake case

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.columns = df.columns.str.replace('__', '_')

df.head()

#### Data visualization and descriptive analysis

In [ ]:
def visualize_hist(columns):
    fig = make_subplots(rows=math.ceil(len(columns) / 2), cols=2)

    for i, col in enumerate(columns):
        row = i // 2 + 1  
        col_num = i % 2 + 1 
        fig.add_histogram(x=df[col], row=row, col=col_num, name=col)

        fig.update_xaxes(title_text=LABELS_DICT[col], row=row, col=col_num)
        fig.update_yaxes(title_text='Frequency', row=row, col=col_num)

    fig.update_layout(height=1400, showlegend=False, template='plotly_dark', title_text="Histograms of Columns")
    fig.show(renderer="png", width=1400, height=2000)

In [ ]:

visualize_hist(df.columns.tolist())

In [ ]:

fig = px.scatter(df, 
                 x='subscription_length', 
                 y='charge_amount', 
                 color='churn', 
                 size='charge_amount',
                 hover_data=['subscription_length'],
                 title='Charge Amount Over Subscription Length with Churn',
                 labels=LABELS_DICT)
fig.show()

fig = px.scatter(df,
                x='seconds_of_use',
                y='frequency_of_use',
                color='churn',
                labels=LABELS_DICT,
                title='Usage vs Frequency')
fig.show()

fig = px.histogram(df,
                    x='customer_value',
                    color='churn',
                    labels=LABELS_DICT,
                    title='Customer value distribution')
fig.show()

fig = px.box(df, 
            x='age_group', 
            y='subscription_length', 
            labels=LABELS_DICT,
            points='all', 
            title='Subscription Length by Age Group')
fig.show()


In [ ]:
correlation_matrix = df.corr()

fig = px.imshow(correlation_matrix,
                labels=LABELS_DICT,
                x=correlation_matrix.index,
                y=correlation_matrix.columns,
                height=600)

fig.show()

### Preprocessing data

In [ ]:
cols_to_drop = ['age']
df = df.drop(columns=cols_to_drop, index=1)

#### Dropping duplicates

In [ ]:
print(df.duplicated().value_counts())

df.drop_duplicates(inplace=True)

#### Removing outliers (IRQ)

In [ ]:
numerical_cols = ['call_failure', 'subscription_length', 'charge_amount',
                     'seconds_of_use', 'frequency_of_use', 'frequency_of_sms', 'distinct_called_numbers', 'customer_value']

def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    data_no_outliers = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    
    return data_no_outliers

for column in numerical_cols:
    df = remove_outliers_iqr(df, column)

#### Feature scalling with MinMaxScaler

In [ ]:
df[numerical_cols] = MinMaxScaler().fit_transform(df[numerical_cols])

##### Categorical encoding (One hot)

In [ ]:
categorical_cols = ['complains', 'tariff_plan', 'status']

df = pd.get_dummies(df, columns=categorical_cols)

In [ ]:
df.head()
df.shape

##### Handling imbalanced classes (SMOTE)

In [ ]:
print("\nClass Distribution for 'churn' Column:")
print(df['churn'].value_counts())


pipeline = Pipeline([
    ('over', SMOTE(sampling_strategy=0.5)),
    ('under', RandomUnderSampler(sampling_strategy=1.0))
])

X_resampled, y_resampled = pipeline.fit_resample(df.drop(target_col, axis=1), df[target_col])
df = pd.concat([X_resampled, y_resampled], axis=1)

print("\nClass Distribution for 'churn' column after SMOTE:")
print( df['churn'].value_counts())

In [ ]:
df.shape

#### Spliting data

In [ ]:
X = df.drop(target_col, axis=1)
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Defining Classifiers

In [ ]:
classifiers = [
    {
        'name': 'SVM',
        'classifier': SVC(),
        'param_grid': {
            # 'classifier__C': [0.1, 1, 10], 
            # 'classifier__kernel': ['linear', 'rbf']
        },
    },
    {
        'name': 'RandomForest',
        'classifier': RandomForestClassifier(),
        'param_grid': {
            'classifier__n_estimators': [50, 100, 200], 
            'classifier__max_depth': [None, 10, 20],
            'regressor__min_samples_leaf': [2, 4, 8],
        },
    },
    {
        'name': 'LogisticRegression',
        'classifier': LogisticRegression(),
        'param_grid': {
            # 'classifier__C': [0.1, 1, 10],
            # 'classifier__penalty': ['l2'],
            # 'classifier__max_iter': [1000],
        },
    },
    {
        'name': 'GradientBoosting',
        'classifier': GradientBoostingClassifier(),
        'param_grid': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__learning_rate': [0.01, 0.1, 0.2],
            'classifier__max_depth': [3, 5, 7],
            # 'regressor__min_samples_split': [2, 4, 8],
            'regressor__min_samples_leaf': [2, 4, 8],
        },
    },
    {
        'name': 'KNeighbors',
        'classifier': KNeighborsClassifier(),
        'param_grid': {
            # 'classifier__n_neighbors': [3, 5, 7],
            # 'classifier__weights': ['uniform', 'distance'],
        },
    },
    {
        'name': 'DecisionTree',
        'classifier': DecisionTreeClassifier(),
        'param_grid': {
            'classifier__criterion': ['gini', 'entropy'],
            'classifier__max_depth': [None, 5, 10],
        },
    },
    {
        'name': 'NaiveBayes',
        'classifier': GaussianNB(),
        'param_grid': {},
    },
    {
        'name': 'AdaBoost',
        'classifier': AdaBoostClassifier(),
        'param_grid': {
            # 'classifier__n_estimators': [50, 100, 200],
            # 'classifier__learning_rate': [0.01, 0.1, 0.2],
        },
    },
        {
        'name': 'ExtraTrees',
        'classifier': ExtraTreesClassifier(),
        'param_grid': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__max_depth': [None, 10, 20],
            'regressor__min_samples_leaf': [2, 4, 8],
        },
    },
    {
        'name': 'Bagging',
        'classifier': BaggingClassifier(),
        'param_grid': {
            # 'classifier__n_estimators': [50, 100, 200],
        },
    },
]



#### Model training and evaluation

In [ ]:
def train_evaluate_model(classifier, X_train, X_test, y_train, y_test):
    pipeline = Pipeline([
        ('pca', PCA(n_components=10)), 
        ('classifier', classifier['classifier']),
    ])

    grid_search = GridSearchCV(pipeline, classifier['param_grid'], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    print(f"{classifier['name']} Test Set Accuracy: {test_accuracy*100:.2f}%")

    result = {
        'name': classifier['name'],
        'test_accuracy': test_accuracy,
        'best_params': best_params,
        'conf_matrix': confusion_matrix(y_test, y_pred),
        'class_report': classification_report(y_test, y_pred),
    }

    return result


def plot_results(results):
    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values(by='test_accuracy', ascending=False)

    fig = px.bar(
        df_results, 
        x='test_accuracy', 
        y='name', 
        orientation='h', 
        title='Test set accuracy of different classifiers',
        color='name',
        labels={'name': 'Classifier'}
    )

    fig.update_layout(xaxis_title='Test Set Accuracy', yaxis_title='Classifier')
    fig.show()

best_overall_model_name = None
best_overall_accuracy = 0.0
results = []

with open(f'{OUTPUT_PATH}model_eval_results.txt', 'w') as file:
    for classifier in classifiers:
        file.write(f"Training and evaluating {classifier['name']}...\n")
        result = train_evaluate_model(classifier, X_train, X_test, y_train, y_test)
        results.append(result)

        if result['test_accuracy'] > best_overall_accuracy:
            best_overall_model_name = classifier['name']
            best_overall_accuracy = result['test_accuracy']

        file.write(f"{classifier['name']} Confusion Matrix:\n{result['conf_matrix']}\n")
        file.write(f"{classifier['name']} Classification Report:\n{result['class_report']}\n")
        file.write(f"Best hyperparameters: {result['best_params']}\n")
        file.write("\n" + "="*50 + "\n")

best_model_result = next(item for item in results if item["name"] == best_overall_model_name)

print("\nOverall best model:")
print(f"Model: {best_overall_model_name}")
print(f"Test Set Accuracy: {best_overall_accuracy*100:.2f}%")

plot_results(results)

In [ ]:
best_model_results = [result for result in results if result['name'] == best_overall_model_name][0]
best_model_params = best_model_results['best_params']
best_classifier = next((classifier for classifier in classifiers if classifier['name'] == best_overall_model_name), None)

if not best_classifier:
    print(f"No information found for the best overall model: {best_overall_model_name}")
    exit()

classifier_params = {key.replace('classifier__', ''): value for key, value in best_model_params.items() if key.startswith('classifier__')}

best_model = best_classifier['classifier'].__class__(**classifier_params)
best_model.fit(X_train, y_train)

nr_features_to_select = 12

selector = RFE(best_model, n_features_to_select=nr_features_to_select, step=1)
selector = selector.fit(X_train, y_train)

selected_features_mask = selector.support_
selected_features = X_train.columns[selected_features_mask]

k = nr_features_to_select 

top_features = selected_features[:k]
top_rankings = selector.ranking_[:k]

sorted_indices = sorted(range(len(top_rankings)), key=lambda k: top_rankings[k], reverse=True)
top_features = [top_features[i] for i in sorted_indices]
top_rankings = [top_rankings[i] for i in sorted_indices]

fig_rfe = px.bar(
    x=top_features,
    y=top_rankings,
    title=f'Recursive Feature Elimination (RFE) Ranking for {best_overall_model_name}',
)

fig_rfe.show()

In [ ]:
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

best_model.fit(X_train_selected, y_train)

y_pred = best_model.predict(X_test_selected)

accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Performance for {best_overall_model_name} with selected features:")
print(f"Accuracy: {accuracy*100:.2f}%")
print("Classification Report:\n", class_report)